In [1]:
import numpy as np
from scipy.stats import spearmanr
import pandas as pd
import seaborn as sns

import urllib.request
import os

from rdkit import Chem, RDLogger, DataStructs
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect, \
                                        GetMACCSKeysFingerprint
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit import RDConfig

In [2]:
def similarity(fps):
    sim = []
    for ind_i, fg in enumerate(fps):
        sim.append([])
        for ind_j in range(0, ind_i + 1):
            if ind_i != ind_j:
                sim[ind_i].append(DataStructs.TanimotoSimilarity(fps[ind_i],fps[ind_j]))
            else:
                sim[ind_i].append(0)
    return sim

In [3]:
def intersection(fps):
    intersec = []
    for ind_i, fg in enumerate(fps):
        intersec.append([])
        for ind_j in range(0, ind_i + 1):
            if ind_i != ind_j:
                andfp = fps[ind_i]&fps[ind_j]
                obl = list(andfp.GetOnBits())
                obls = [fcat.GetEntryDescription(item) for item in obl]
                intersec[ind_i].append(obls)
            else:
                intersec[ind_i].append([])
    return intersec

In [4]:
def mol_drawer(sim_func, fps, l_mol_objs, n):
    sim = sim_func(fps) #counting similarity scores
    """
    Creating a list of tuples with simularity /
        score and indexes of molecules in i_mol_objs

    """
    max_sim = []
    for i in range(len(sim)):
        max_sim.append((sim[i][np.argmax(sim[i])],i,np.argmax(sim[i])))
    max_sim = sorted(max_sim)
    """
    Extracting 4 pairs of most similar molecules / 
         from sorted l_mol_objs according to their /
             similarity score to be visialized

    """
    out = []
    for i in range(n):
        out.append(l_mol_objs[max_sim[len(max_sim)-1-i][1]])
        out.append(l_mol_objs[int(max_sim[len(max_sim)-1-i][2])])
        print(max_sim[len(max_sim)-1-i])
    return max_sim, Draw.MolsToGridImage(out, molsPerRow=2)

In [5]:
def request(cids):
    molecules = []
    for cid in cids:
        if cid != '_':
            url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + str(cid) + '/property/CanonicalSMILES/csv'
            response = urllib.request.urlopen(url)
            molecules.append(response.read().decode('utf-8'))
    list_of_molecules = []
    for ind, mol in enumerate(molecules):
        a = mol.split(sep='"')
        molecules[ind] = a[5]
        list_of_molecules.append([chemicals['name'][ind], int(a[4][1:-1]), a[5]])
    return molecules, list_of_molecules

In [6]:
chemicals = pd.read_csv("direct_reprogramming_non-genetics.csv")
chemicals

,cid,link,name,Synonyms,MOA
0,459803,https://pubchem.ncbi.nlm.nih.gov/compound/459803,AC1LA18U,NaN,Inhibitor of the HIF prolyl 4-hydroxylase
1,91899426,https://pubchem.ncbi.nlm.nih.gov/compound/9189...,2-phospho-L-ascorbic acid,2-phospho-L-ascorbic acid;BDBM92477,_
2,286003,https://pubchem.ncbi.nlm.nih.gov/compound/286003,DTXSID80301486,NaN,Agonist of the Adenosine Receptor
3,47289,https://pubchem.ncbi.nlm.nih.gov/compound/47289,4-(Methylnitrosamino)-1-(3-pyridyl)-1-butanone,NaN,_
4,451668,https://pubchem.ncbi.nlm.nih.gov/compound/451668,5-Aza-2'-deoxycytidine,Decitabine;5-Aza-2'-deoxycytidine;2353-33-5;Da...,Inhibitor of DNA methylation
...,...,...,...,...,...
153,2814138,https://pubchem.ncbi.nlm.nih.gov/compound/2814138,OAC2,NaN,_
154,5282411,https://pubchem.ncbi.nlm.nih.gov/compound/5282411,Prostaglandin I2,NaN,"Activator of prostaglandin receptors EP1, EP2,..."
155,10202642,https://pubchem.ncbi.nlm.nih.gov/compound/1020...,GW788388,NaN,Inhibitor of Transforming Growth Factor Beta S...
156,6437836,https://pubchem.ncbi.nlm.nih.gov/compound/6437836,Peretinoin,NaN,Agonist of retinoid acid receptor (RAR)


In [ ]:
cids = chemicals['cid']
molecules, list_of_molecules = request(cids)

In [ ]:
molecules

In [ ]:
l_mol_objs = [Chem.MolFromSmiles(molecule) for molecule in molecules]

In [ ]:
Draw.MolsToGridImage(l_mol_objs, molsPerRow=2)

In [ ]:
fps = [GetMorganFingerprintAsBitVect(molecule, 2) for molecule in l_mol_objs]

In [ ]:
sim = similarity(fps)

In [ ]:
max_sim, pictures = mol_drawer(similarity, fps, l_mol_objs, 4)

In [ ]:
pictures

In [ ]:
print(list_of_molecules[126][1], list_of_molecules[128][1])

In [ ]:
fName=os.path.join(RDConfig.RDDataDir,'FunctionalGroups.txt')
from rdkit.Chem import FragmentCatalog
fparams = FragmentCatalog.FragCatParams(1,6,fName)
fparams.GetNumFuncGroups()

fcat=FragmentCatalog.FragCatalog(fparams)
fcgen=FragmentCatalog.FragCatGenerator()
m = Chem.MolFromSmiles('OCC=CC(=O)O')
fcgen.AddFragsFromMol(m,fcat)

In [ ]:
fcat=FragmentCatalog.FragCatalog(fparams)
for molecule in l_mol_objs: 
    nAdded=fcgen.AddFragsFromMol(molecule,fcat)
print(fcat.GetNumEntries())
fcat.GetEntryDescription(523)

In [ ]:
fpgen = FragmentCatalog.FragFPGenerator()
frag_fps = [fpgen.GetFPForMol(molecule,fcat) for molecule in l_mol_objs]

In [ ]:
intersections = intersection(frag_fps)
intersections

In [ ]:
pd.DataFrame(intersections)

# Creating a Table

In [ ]:
reprogramming_data = pd.read_csv("direct_reprogramming_non-genetics_structure.csv")
reprogramming_data

In [ ]:
out_table = []
for ind, chem in enumerate(reprogramming_data["name of chemical 1,CID 1;name of chemical 2,CID 2"]):
    print(ind)
    chem = chem.split(sep=';')
    cids = []
    for mol in chem:
        mol = mol.split(sep=',')
        if mol[1] != "_":
            cids.append(int(mol[1]))
    molecules, _ =request(cids)
    out_table.append([reprogramming_data["Source Cell Type"][ind], reprogramming_data["Target Cell Type"][ind], ';'.join(molecules)])

In [ ]:
pd.DataFrame(out_table)

In [ ]:
pd.write_csv(out_table)